In [15]:
import os
from dotenv import load_dotenv
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas

# Load the .env file
load_dotenv()

# Access the variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# OPENAI_API_KEY



In [3]:
embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
max_tokens = 8000  # the maximum for text-embedding-3-small is 8191

In [14]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string("Око њега се развијају приче о историјским личностима и безименим ликовима који су плод пишчеве маште. Роман На Дрини ћуприја, понекад називан и Вишеградска ", "cl100k_base")

99

In [10]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [9]:
from openai import OpenAI
client = OpenAI()

def normalize_l2(x):
    x = np.array(x)
    if x.ndim == 1:
        norm = np.linalg.norm(x)
        if norm == 0:
            return x
        return x / norm
    else:
        norm = np.linalg.norm(x, 2, axis=1, keepdims=True)
        return np.where(norm == 0, x, x / norm)
    


def get_embedding(text, model="text-embedding-3-small", dimension=None):
    text = text.replace("\n", " ")

    embedded_text = client.embeddings.create(input = [text], model=model).data[0].embedding
    if not dimension:
       return embedded_text
    cut_dim = embedded_text[:dimension]
    return normalize_l2(cut_dim)


embedding = get_embedding("recenica akldakjdn jakjaaj jjjjjj", model=embedding_model, dimension=768)
len(embedding)
# 1536
# 768 je outoutovih srpskih
# zakljucak, treba redukcija dimenzionalnosti

768

In [21]:
loaded_table = pq.read_table("datasets/dataset_processed.parquet")
df = loaded_table.to_pandas()
first_n_rows = df.head(15)
# first_n_rows
len(first_n_rows)

15

In [24]:
def search_reviews(df, product_description, n=3, pprint=True):
   embedding = get_embedding(product_description, model=embedding_model)
   df['similarities'] = df.ada_embedding.apply(lambda x: cosine_similarity(x, embedding))
   res = df.sort_values('similarities', ascending=False).head(n)
   return res

# Example of creating an 'ada_embedding' column
first_n_rows['ada_embedding'] = first_n_rows['context'].apply(lambda x: get_embedding(x, model=embedding_model))
res = search_reviews(first_n_rows, 'Šta je ispred Glavne zgrade Notr Dame?', n=3)


/tmp/ipykernel_8028/3373473675.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_n_rows['ada_embedding'] = first_n_rows['context'].apply(lambda x: get_embedding(x, model=embedding_model))
/tmp/ipykernel_8028/3373473675.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['similarities'] = df.ada_embedding.apply(lambda x: cosine_similarity(x, embedding))


In [25]:
res

,title,context,queries,ada_embedding,similarities
0,University_of_Notre_Dame,"Arhitektonski, škola ima katolički karakter. N...","[Kome se u 1858. godini u Lurdu, Francuska, na...","[-0.04610486701130867, 0.00042641302570700645,...",0.575776
7,University_of_Notre_Dame,Bibliotečki sistem univerziteta podeljen je iz...,[Koliko je ništa visine glavna biblioteka u No...,"[-0.049503572285175323, -0.019839294254779816,...",0.568562
2,University_of_Notre_Dame,Univerzitet je glavna sedište Kongregacije Sve...,"[Gde je sedište Kongregacije Svetog Krsta?, Št...","[-0.04491373524069786, 0.035998500883579254, 0...",0.441146
